In [1]:
import pandas as pd
from datetime import datetime

In [4]:
!pwd

/Users/paulina/Documents/BBCProject/merged


In [60]:
# Read file
alj = pd.read_csv("../alj/AllArticlesContent.csv")

# add columns for indexing
alj['location'] = "Palestine"
alj['press'] = "Al Jazeera"

# Filter column of alj to keep only relevant articles.
keywords = 'Israel|Palestine|Gaza|Ghaza'
alj = alj[alj['Content'].str.contains(keywords, case=False, na=False)]

# Extract the date using regex and create a new column 'date'
# The regex pattern \d{4}/\d{1,2}/\d{1,2} matches the date format YYYY/MM/DD
alj['Timestamp'] = alj['URL'].str.extract(r'/(\d{4}/\d{1,2}/\d{1,2})')[0]
alj['Timestamp'] = pd.to_datetime(alj['Timestamp'])
alj['Timestamp'] = alj['Timestamp'].dt.strftime('%Y/%m/%d')

# Rename
alj.rename(columns={'Content': 'Text'}, inplace=True)

# Drop columns that are not needed
alj.drop(['Subtitle', 'URL'], axis=1, inplace=True)

# Order the columns
alj = alj[['Timestamp', 'Title', 'Text', 'location', 'press']]

alj.head()

,Timestamp,Title,Text,location,press
0,2024/02/12,"Israel kills dozens in Rafah strikes, frees tw...",Israel has carried out air strikes in the sout...,Palestine,Al Jazeera
1,2024/02/11,Hamas warns Israeli invasion of Rafah will ‘to...,Hamas has warned Israel thata ground offensive...,Palestine,Al Jazeera
2,2024/02/11,Palestinian President Mahmoud Abbas in Qatar f...,Palestinian President Mahmoud Abbas has arrive...,Palestine,Al Jazeera
3,2024/02/11,What’s happening in Gaza’s Rafah as Israel thr...,"Rafah, a looming Israeli ground “operation”, a...",Palestine,Al Jazeera
4,2024/02/11,UNRWA refutes Israeli implications of Hamas tu...,The chief of the UN’s aid agency in Gaza has s...,Palestine,Al Jazeera


In [61]:
# Read file
ukr = pd.read_excel("../BBCUkraine/Ukraine.xlsx")
ukr['location'] = "Ukraine"
ukr['press'] = "BBC"

# Convert the string to a datetime object
ukr['Timestamp'] = pd.to_datetime(ukr['Timestamp']).dt.strftime('%Y/%m/%d')

# Clean up
ukr.drop(['Authors'], axis=1, inplace=True)
ukr.head()

,Timestamp,Title,Text,location,press
0,2024/02/17,Ukraine war: 'Artificial shortage' of weapons ...,Ukraine's President Volodymyr Zelensky has mad...,Ukraine,BBC
1,2024/02/17,Avdiivka: Ukraine troops leave embattled easte...,Ukraine says its troops have withdrawn from Av...,Ukraine,BBC
2,2024/02/17,Ukraine war: Is Avdiivka's fall a sign Russia ...,"In order to preserve life and encirclement, I ...",Ukraine,BBC
3,2024/02/17,Biden condemns House taking break without pass...,President Joe Biden has said the US House's de...,Ukraine,BBC
4,2024/02/16,Ukraine Russia war: US warns Avdiivka could fa...,The US has warned that Russia could seize Ukra...,Ukraine,BBC


In [62]:
# Read file
bbcme = pd.read_excel("../BBCME/MiddleEast.xlsx")
bbcme['location'] = "Palestine"
bbcme['press'] = "BBC"

# Convert the string to a datetime object
bbcme['Timestamp'] = pd.to_datetime(bbcme['Timestamp']).dt.strftime('%Y/%m/%d')

# Clean up
bbcme.drop(['Authors'], axis=1, inplace=True)
bbcme.head()

,Timestamp,Title,Text,location,press
0,2024/02/16,Israel Gaza war: Satellite images show constru...,Satellite images appear to show extensive cons...,Palestine,BBC
1,2024/02/16,Southend West Lib Dem quits over 'offensive' s...,A woman who was expected to become the next Li...,Palestine,BBC
2,2024/02/16,Gaza's Nasser hospital: Fears for patients as ...,The Israeli military says its special forces a...,Palestine,BBC
3,2024/02/16,Hamas hostages: Stories of the people taken fr...,Israel's military says that two male hostages ...,Palestine,BBC
4,2024/02/15,Nasser hospital in 'catastrophic' condition as...,"Israel's military claims it has captured ""doze...",Palestine,BBC


In [63]:
for i in [alj, bbcme, ukr]:
    print(i.columns)

Index(['Timestamp', 'Title', 'Text', 'location', 'press'], dtype='object')
Index(['Timestamp', 'Title', 'Text', 'location', 'press'], dtype='object')
Index(['Timestamp', 'Title', 'Text', 'location', 'press'], dtype='object')


In [64]:
# Merge and save all the data in a csv
news = pd.concat([alj, bbcme, ukr], axis=0, ignore_index=True)
news.to_csv("allNewsMerged.csv", index=False)

In [86]:
# Separte to get wide format based on location and press
pal_news =news.loc[news['location'] == "Palestine"]
bbc_news =news.loc[news['press'] == "BBC"]

In [92]:
# Remove redundatn information from the title
pal_news['Title'] = pal_news['Title'].str.replace(" - BBC News", "", regex=False)

/var/folders/n5/y53yj96j3d7bsb54xrzys8s40000gn/T/ipykernel_11678/3577342679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pal_news['Title'] = pal_news['Title'].str.replace(" - BBC News", "", regex=False)


In [93]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Separate titles by press
df_bbc = pal_news[pal_news['press'] == 'BBC']
df_alj = pal_news[pal_news['press'] == 'Al Jazeera']

# Vectorize titles
vectorizer = TfidfVectorizer()
tfidf_bbc = vectorizer.fit_transform(df_bbc['Title'])
# Use the same vectorizer to ensure the same feature space
tfidf_alj = vectorizer.transform(df_alj['Title'])

# Compute Cosine Similarity
cosine_sim = cosine_similarity(tfidf_bbc, tfidf_alj)

# Create a DataFrame to log results
# Generating pairs of titles from different press types with their similarity scores
results = []
for i, title_bbc in enumerate(df_bbc['Title']):
    for j, title_alj in enumerate(df_alj['Title']):
        results.append({
            'BBC_Title': title_bbc,
            'Alj_Title': title_alj,
            'Similarity': cosine_sim[i, j]
        })
results_df = pd.DataFrame(results)

print(results_df)


                                                BBC_Title  \
0       Israel Gaza war: Satellite images show constru...   
1       Israel Gaza war: Satellite images show constru...   
2       Israel Gaza war: Satellite images show constru...   
3       Israel Gaza war: Satellite images show constru...   
4       Israel Gaza war: Satellite images show constru...   
...                                                   ...   
990787  Biden to visit Israel as concerns grow over co...   
990788  Biden to visit Israel as concerns grow over co...   
990789  Biden to visit Israel as concerns grow over co...   
990790  Biden to visit Israel as concerns grow over co...   
990791  Biden to visit Israel as concerns grow over co...   

                                                Alj_Title  Similarity  
0       Israel kills dozens in Rafah strikes, frees tw...    0.022227  
1       Hamas warns Israeli invasion of Rafah will ‘to...    0.000000  
2       Palestinian President Mahmoud Abbas in Qata

In [94]:
# Sort the results by similarity
results_df.sort_values(by='Similarity', ascending=False, inplace=True)